# México COVID-19 DataBase

In [1]:
import pandas as pd
import numpy as np
from scipy import integrate
from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error, median_absolute_error
from scipy.integrate import odeint
from scipy.optimize import differential_evolution, minimize
import matplotlib.pyplot as plt
import PDEparams as pde

In [2]:
data = pd.read_csv('201220COVID19MEXICO.csv',encoding = 'latin_1')

In [3]:
data

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2020-12-20,002371,1,4,3,2,3,3,8,1,...,99,1,1,2,97,3,99,MÃ©xico,97,97
1,2020-12-20,11fb00,1,12,9,2,9,9,15,1,...,1,1,1,2,97,3,99,MÃ©xico,97,97
2,2020-12-20,092521,1,12,9,1,9,9,17,1,...,2,1,1,2,97,3,99,MÃ©xico,97,97
3,2020-12-20,0955a5,2,12,6,1,14,6,10,1,...,2,1,1,2,97,3,99,MÃ©xico,97,97
4,2020-12-20,1a1f12,2,4,14,2,9,14,67,2,...,2,1,1,2,97,3,99,MÃ©xico,97,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3377512,2020-12-20,412c07,1,12,9,2,9,9,17,1,...,1,2,97,1,2,7,99,MÃ©xico,97,97
3377513,2020-12-20,40f8d9,1,12,13,2,13,13,69,1,...,2,2,97,2,97,6,99,MÃ©xico,97,97
3377514,2020-12-20,3a8f5a,2,12,9,1,9,9,15,1,...,1,2,97,1,2,7,99,MÃ©xico,97,97
3377515,2020-12-20,365280,1,12,9,2,9,9,10,1,...,1,2,97,1,1,3,99,MÃ©xico,97,97


## Características

In [4]:
count = 0
for a in data:
    print(a)
    count = count + 1
print('En total hay:', count, 'características.')

FECHA_ACTUALIZACION
ID_REGISTRO
ORIGEN
SECTOR
ENTIDAD_UM
SEXO
ENTIDAD_NAC
ENTIDAD_RES
MUNICIPIO_RES
TIPO_PACIENTE
FECHA_INGRESO
FECHA_SINTOMAS
FECHA_DEF
INTUBADO
NEUMONIA
EDAD
NACIONALIDAD
EMBARAZO
HABLA_LENGUA_INDIG
INDIGENA
DIABETES
EPOC
ASMA
INMUSUPR
HIPERTENSION
OTRA_COM
CARDIOVASCULAR
OBESIDAD
RENAL_CRONICA
TABAQUISMO
OTRO_CASO
TOMA_MUESTRA_LAB
RESULTADO_LAB
TOMA_MUESTRA_ANTIGENO
RESULTADO_ANTIGENO
CLASIFICACION_FINAL
MIGRANTE
PAIS_NACIONALIDAD
PAIS_ORIGEN
UCI
En total hay: 40 características.


## Casos confirmados

In [5]:
confirmados = 0
for a in data.loc[:,'RESULTADO_LAB']:
    if a == 1:
        confirmados = a + confirmados
    else:
        continue
print('Hay un total de',confirmados,'confirmados.')        

Hay un total de 1215208 confirmados.


### Se hace una base de datos solamente de los confirmados

In [6]:
%%time
lst = []
for n in range(len(data)-1):
    if data.loc[n,'RESULTADO_LAB']==1 :
        lst.append(data.loc[n,:])        
df_confirmados = pd.DataFrame(lst)   
df_confirmados.reset_index(drop=True, inplace=True)

CPU times: user 6min 3s, sys: 9.25 s, total: 6min 12s
Wall time: 6min 18s


In [7]:
df_confirmados

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2020-12-20,002371,1,4,3,2,3,3,8,1,...,99,1,1,2,97,3,99,MÃ©xico,97,97
1,2020-12-20,11fb00,1,12,9,2,9,9,15,1,...,1,1,1,2,97,3,99,MÃ©xico,97,97
2,2020-12-20,092521,1,12,9,1,9,9,17,1,...,2,1,1,2,97,3,99,MÃ©xico,97,97
3,2020-12-20,0955a5,2,12,6,1,14,6,10,1,...,2,1,1,2,97,3,99,MÃ©xico,97,97
4,2020-12-20,1a1f12,2,4,14,2,9,14,67,2,...,2,1,1,2,97,3,99,MÃ©xico,97,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215203,2020-12-20,38a57c,2,12,8,2,10,8,37,1,...,2,1,1,2,97,3,99,MÃ©xico,97,97
1215204,2020-12-20,366b78,1,4,9,1,15,15,70,1,...,2,1,1,2,97,3,99,MÃ©xico,97,97
1215205,2020-12-20,1f485c,2,12,18,1,18,18,17,1,...,1,1,1,2,97,3,99,MÃ©xico,97,97
1215206,2020-12-20,2e2f64,2,4,9,1,2,9,6,1,...,2,1,1,2,97,3,99,MÃ©xico,97,97


# Datos interesantes pero no necesarios para el proyecto de ML

In [8]:
muertos = 0
for a in df_confirmados.loc[:,'FECHA_DEF']:
    if a != '9999-99-99':
        muertos = muertos + 1
print('Hay',muertos,'muertos.')

Hay 113087 muertos.


In [9]:
hom = 0
muj = 0
noid = 0
for a in df_confirmados.loc[:,'SEXO']:
    if a == 1:
        muj = muj + 1
    if a == 2:
        hom = hom + 1
    else:
        noid = noid + 0
print('De las', confirmados,'gente contagiada hay \n', muj, 'mujeres \n',hom,'hombres \n',noid,'no especificados.')

De las 1215208 gente contagiada hay 
 598754 mujeres 
 616454 hombres 
 0 no especificados.


In [10]:
hom = 0
muj = 0
noid = 0
for a in range(len(df_confirmados.loc[:,'SEXO'])):
    if df_confirmados.loc[a,'SEXO'] == 1 and df_confirmados.loc[a,'FECHA_DEF'] != '9999-99-99':
        muj = muj + 1
    if df_confirmados.loc[a,'SEXO'] == 2 and df_confirmados.loc[a,'FECHA_DEF'] != '9999-99-99':
        hom = hom + 1
print('Y de los',muertos,' muertos son \n', muj, 'mujeres \n',hom,'hombres')

Y de los 113087  muertos son 
 41358 mujeres 
 71729 hombres


In [11]:
diab = 0
for a in range(len(df_confirmados.loc[:,'DIABETES'])):
    if df_confirmados.loc[a,'DIABETES'] == 1 and df_confirmados.loc[a,'FECHA_DEF'] != '9999-99-99':
        diab = diab + 1
print('De las',muertos,'personas muertas,', diab, 'tenian Diabetes.')

De las 113087 personas muertas, 43691 tenian Diabetes.


In [12]:
hiper = 0
for a in range(len(df_confirmados.loc[:,'DIABETES'])):
    if df_confirmados.loc[a,'HIPERTENSION'] == 1 and df_confirmados.loc[a,'FECHA_DEF'] != '9999-99-99':
        hiper = hiper + 1
print('De las',muertos,'personas muertas,', hiper, 'tenian Hipertensión.')

De las 113087 personas muertas, 52068 tenian Hipertensión.


In [13]:
hiperdiab = 0
for a in range(len(df_confirmados.loc[:,'DIABETES'])):
    if df_confirmados.loc[a,'HIPERTENSION'] == 1 and df_confirmados.loc[a,'DIABETES'] == 1 and df_confirmados.loc[a,'FECHA_DEF'] != '9999-99-99':
        hiperdiab  = hiperdiab  + 1
print('De las',muertos,'personas muertas,', hiperdiab, 'tenian Hipertensión y Diabetes.')

De las 113087 personas muertas, 28703 tenian Hipertensión y Diabetes.


In [14]:
fumaban = 0
for a in range(len(df_confirmados.loc[:,'DIABETES'])):
    if df_confirmados.loc[a,'TABAQUISMO'] == 1 and df_confirmados.loc[a,'FECHA_DEF'] != '9999-99-99':
        fumaban  = fumaban  + 1
print('De las',muertos,'personas muertas,', fumaban, 'fumaban.')

De las 113087 personas muertas, 9083 fumaban.


In [15]:
obeso = 0
for a in range(len(df_confirmados.loc[:,'DIABETES'])):
    if df_confirmados.loc[a,'OBESIDAD'] == 1 and df_confirmados.loc[a,'FECHA_DEF'] != '9999-99-99':
        obeso  = obeso  + 1
print('De las',muertos,'personas muertas,', obeso, 'eran obesas.')

De las 113087 personas muertas, 26867 eran obesas.


In [196]:
print(round(diab/muertos*100,2),'% de personas muertas tenían Diabetes,\n',
      round(hiper/muertos*100,2),'% de personas muertas tenían Hipertensión, \n',
     round(hiperdiab/muertos*100,2),'% de personas muertas tenían Hipertensión y Diabetes, \n',
     round(obeso/muertos*100,2),'% de personas muertas eran obesos. \n',
        round(fumaban/muertos*100,2),'% de personas muertas fumaban. \n')

38.63 % de personas muertas tenían Diabetes,
 46.04 % de personas muertas tenían Hipertensión, 
 25.38 % de personas muertas tenían Hipertensión y Diabetes, 
 23.76 % de personas muertas eran obesos. 
 8.03 % de personas muertas fumaban. 



# Datos importantes para el proyecto

## Casos y muertes acumuladas

In [17]:
%%time
countmuertes = 0
dictmuertes = dict()
for a in df_confirmados.loc[:,'FECHA_DEF']:
    if a == '9999-99-99':
        continue
    else:    
        dictmuertes[a] = dictmuertes.get(a,0) + 1
    
lst = list()
for (k,v) in dictmuertes.items():
    newtup = (k,v)
    lst.append(newtup)
lstmuert = sorted(lst)
#print(lst)

#for v,k in lstmuert:
 #   print(v,k)

CPU times: user 6.55 s, sys: 1.68 s, total: 8.23 s
Wall time: 8.41 s


In [18]:
%%time
countinfect = 0
dictinfect = dict()
for a in df_confirmados.loc[:,'FECHA_INGRESO']:
    dictinfect[a] = dictinfect.get(a,0) + 1

lst = list()
for (k,v) in dictinfect.items():
    newtup = (k,int(v))
    lst.append(newtup)
lstinfect = sorted(lst)
#print(lst)

#for v,k in lstinfect:
 #   print(v,k)

CPU times: user 395 ms, sys: 4.75 ms, total: 399 ms
Wall time: 402 ms


### Tabla de muertos por día y acumulados

In [192]:
muer_dia = pd.DataFrame(lstmuert)
# le quito la fecha porque lo reconoce como string y marcaba un error en el modelo
muertdia_sinfecha = muer_acum.drop([0], axis = 1)
muer_dia

,0,1
0,2020-03-18,2
1,2020-03-20,1
2,2020-03-22,1
3,2020-03-23,1
4,2020-03-24,2
...,...,...
270,2020-12-15,425
271,2020-12-16,384
272,2020-12-17,285
273,2020-12-18,188


In [193]:
muertos_acumulados = []
total = 0
for a in range(len(muertdia_sinfecha[1])):
    total = total + muertdia_sinfecha[1][a]
    muertos_acumulados.append(total)
muertos_acumulados = pd.DataFrame(muertos_acumulados)
muertos_acumulados

,0
0,2
1,3
2,4
3,5
4,7
...,...
270,112164
271,112548
272,112833
273,113021


### Tabla de infectados por día y acumulados

In [180]:
infect_dia = pd.DataFrame(lstinfect)
# le quito la fecha porque lo reconoce como string y marcaba un error en el modelo
infectdia_sinfecha = infect_acum.drop([0], axis = 1)
infect_dia

,0,1
0,2020-01-13,1
1,2020-01-28,1
2,2020-01-29,1
3,2020-02-24,2
4,2020-02-27,4
...,...,...
294,2020-12-16,4534
295,2020-12-17,3112
296,2020-12-18,1429
297,2020-12-19,90


In [176]:
infacumulados = []
total = 0
for a in range(len(infectdia_sinfecha[1])):
    total = total + infectdia_sinfecha[1][a]
    infacumulados.append(total)
infacumulados = pd.DataFrame(infacumulados)
infacumulados

,0
0,1
1,2
2,3
3,5
4,9
...,...
294,1210576
295,1213688
296,1215117
297,1215207


## Modelo SIR

In [212]:
def SIR(z, t, be, gm):
    '''The input z corresponds to the current state of the system, z = [x, y]. Since the input is in 1D, no 
    pre-processing is needed.
    
    t is the current time.
    
    a and b correspond to the unknown parameters.
    '''
    
    S, I ,R = z
    
    return [-be*(S*I), be*(S*I)-gm*I, gm*I]

In [213]:
def init_I():
    return 1

def init_R():
    return 1

def init_S():
    return 120000000

In [223]:
# Se toman las bases 'sin fecha' o las acumuladas
#estas son:
#infectdia_sinfecha
#infacumulados


my_model = pde.PDEmodel(infacumulados, SIR, [init_S, init_R, init_I], bounds=[(-2, 2), (-2,2)], 
                        param_names=[r'$be$', r'$gm$'], nvars=3, ndims=0, 
                        nreplicates=1, obsidx=[0,2] , outfunc = None)
help(pde.PDEmodel)

Help on class PDEmodel in module PDEparams:

class PDEmodel(builtins.object)
 |  PDEmodel(data, model, initfunc, bounds, param_names=None, nvars=1, ndims=1, nreplicates=1, obsidx=None, outfunc=None)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, data, model, initfunc, bounds, param_names=None, nvars=1, ndims=1, nreplicates=1, obsidx=None, outfunc=None)
 |      Initialises the PDEmodel object.
 |      
 |      Parameters
 |      ----------
 |      data: DataFrame of data points with each entry in the form:
 |      
 |                              [timepoint, coordinates, fuction values]
 |      
 |      model: the PDE model to fit to the data. Should accept the parameters to
 |             estimate as its last inputs and return the time derivatives of the
 |             functions.
 |      
 |      initfunc: array of functions defining the initial conditions for the model.
 |      
 |      bounds: the contraints for the parameter values to use in the estimation,
 |              a

In [224]:
my_model.initial_condition

array([120000000,         1,         1])

In [225]:
%%time
my_model.fit()

       $be$      $gm$
0 -0.699017  0.485927
CPU times: user 8min 57s, sys: 7 s, total: 9min 4s
Wall time: 9min 57s


In [226]:
my_model.best_params

,$be$,$gm$
0,-0.699017,0.485927


In [227]:
my_model.best_error

inf